<a href="https://colab.research.google.com/github/Richik06/Richik06/blob/main/INVOICE_CATEGORIZER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import joblib

# Sample training data
data = {
    "text": [
        "Zomato order chicken biryani",
        "Swiggy delivery paneer tikka",
        "Uber ride to airport",
        "Ola cab ride",
        "Indigo flight to Mumbai",
        "BSNL broadband bill",
        "Electricity bill",
        "Amazon purchase headphone",
        "Flipkart shopping mobile",
        "Recharge Airtel prepaid",
        "Dominos pizza order",
        "Train ticket IRCTC",
        "Metro smartcard recharge",
        "Gas cylinder BharatGas"
    ],
    "category": [
        "Food", "Food", "Travel", "Travel", "Travel", "Utilities", "Utilities",
        "Shopping", "Shopping", "Utilities", "Food", "Travel", "Travel", "Utilities"
    ]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Step 1: Text vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])  # Features
y = df['category']                        # Labels

# Step 2: Train classifier
model = MultinomialNB()
model.fit(X, y)

# Step 3: Save model and vectorizer
joblib.dump(model, "model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")

print("✅ Model and vectorizer saved as 'model.pkl' and 'vectorizer.pkl'")


✅ Model and vectorizer saved as 'model.pkl' and 'vectorizer.pkl'


In [5]:
import streamlit as st
import pandas as pd
import easyocr
import joblib
import tempfile
import os
from pdf2image import convert_from_path
from PIL import Image

# ---------- Load model and vectorizer ----------
@st.cache_resource
def load_model():
    model = joblib.load("model.pkl")
    vectorizer = joblib.load("vectorizer.pkl")
    return model, vectorizer

model, vectorizer = load_model()

# ---------- OCR Function ----------
def run_ocr(file_path):
    reader = easyocr.Reader(['en'])
    text_lines = reader.readtext(file_path, detail=0)
    return text_lines

# ---------- File Upload ----------
st.title("🧾 Invoice/Bill Digitizer & Categorizer")
st.write("Upload your invoice image or PDF to auto-categorize the line items into Food, Travel, Utilities, etc.")

uploaded_file = st.file_uploader("Upload Invoice", type=["jpg", "jpeg", "png", "pdf"])

if uploaded_file:
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        tmp_file.write(uploaded_file.read())
        temp_path = tmp_file.name

    # Convert PDF to image
    if uploaded_file.name.lower().endswith(".pdf"):
        pages = convert_from_path(temp_path)
        image_path = temp_path + ".jpg"
        pages[0].save(image_path, 'JPEG')
    else:
        image_path = temp_path

    # ---------- OCR + Prediction ----------
    st.info("🔍 Running OCR on your invoice...")
    ocr_lines = run_ocr(image_path)

    if len(ocr_lines) == 0:
        st.warning("No text found in invoice.")
    else:
        st.success("✅ Text successfully extracted from invoice.")

        # Predict categories
        X_test = vectorizer.transform(ocr_lines)
        preds = model.predict(X_test)

        # Combine results
        df = pd.DataFrame({"Line Item": ocr_lines, "Predicted Category": preds})
        st.dataframe(df)

        # Downloadable CSV
        csv = df.to_csv(index=False).encode('utf-8')
        st.download_button(
            label="📥 Download Categorized Data as CSV",
            data=csv,
            file_name='categorized_invoice.csv',
            mime='text/csv',
        )


2025-07-25 07:33:13.369 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 07:33:13.371 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 07:33:13.372 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 07:33:13.373 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 07:33:13.377 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 07:33:13.378 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 07:33:13.379 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 07:33:13.381 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [2]:
%pip install streamlit pandas easyocr joblib pdf2image Pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 394.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/

In [6]:
import os

# List the saved files
os.listdir()


['.config', 'model.pkl', 'vectorizer.pkl', 'sample_data']

In [7]:
print("✅ Model and vectorizer saved!")
print("📁 Files in directory:", os.listdir())


✅ Model and vectorizer saved!
📁 Files in directory: ['.config', 'model.pkl', 'vectorizer.pkl', 'sample_data']


In [8]:
from google.colab import files
files.download("model.pkl")
files.download("vectorizer.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>